<a href="https://colab.research.google.com/github/da-head0/spark-example/blob/main/Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 환경 구축

- 로컬 환경에서 PySpark 시도시 오류가 나서 코랩으로 진행하였습니다.

In [5]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar xf spark-2.4.8-bin-hadoop2.7.tgz
!pip install -q findspark
import os

# 자바, 스파크 환경설정
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"
import findspark
findspark.init()

## 데이터셋 정보

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("Learning_Spark").getOrCreate() # 세션을 구성
data = spark.read.csv("pet_food_customer_orders.csv", inferSchema=True, header=True)

- 데이터셋은 제가 다음 개인 프로젝트 주제로 계획중인 애완동물 사료 쇼핑몰 판매 데이터입니다.

- 건식, 습식 사료 판매 데이터를 가지고 습식을 살 고객을 예측하는 목적의 데이터인데요, 오늘은 일단 스파크와 하둡의 사용방법을 배우는 것을 목표로 하기 때문에 해당 챌린지를 예측하지는 않겠습니다.

In [10]:
# 데이터의 길이, 행 갯수
data.count(), len(data.columns)

(49042, 36)

In [12]:
# 데이터프레임 보기
data.show(10) # 숫자가 없으면 처음 20개를 출력

+--------------------+--------------------+----------------+---------------------+----------------------------------+---------------------------+-------------+--------------------+-----------------+--------------------+---------------------+--------+------+--------------+--------------+----------------------+------------------------+-----------------------+-------------------+-----------+--------+----------------+---------+-------------------------+-------------+-------------------+------------------+--------------------------+------------------+-----------------------------------+------------------------+-----------------------------------------+----------------------+---------------------------------------+---------------------------------+--------------------------------+
|         customer_id|              pet_id|pet_order_number|wet_food_order_number|orders_since_first_wet_trays_order|pet_has_active_subscription|pet_food_tier| pet_signup_datetime|pet_allergen_list|pet_fav_flavour_

In [13]:
# 데이터프레임을 구성하는 정보 확인
data.printSchema()

root
 |-- customer_id: decimal(20,0) (nullable = true)
 |-- pet_id: decimal(20,0) (nullable = true)
 |-- pet_order_number: integer (nullable = true)
 |-- wet_food_order_number: double (nullable = true)
 |-- orders_since_first_wet_trays_order: double (nullable = true)
 |-- pet_has_active_subscription: boolean (nullable = true)
 |-- pet_food_tier: string (nullable = true)
 |-- pet_signup_datetime: timestamp (nullable = true)
 |-- pet_allergen_list: string (nullable = true)
 |-- pet_fav_flavour_list: string (nullable = true)
 |-- pet_health_issue_list: string (nullable = true)
 |-- neutered: boolean (nullable = true)
 |-- gender: string (nullable = true)
 |-- pet_breed_size: string (nullable = true)
 |-- signup_promo: string (nullable = true)
 |-- ate_wet_food_pre_tails: boolean (nullable = true)
 |-- dry_food_brand_pre_tails: string (nullable = true)
 |-- pet_life_stage_at_order: string (nullable = true)
 |-- order_payment_date: timestamp (nullable = true)
 |-- kibble_kcal: double (nulla

## 데이터셋 분석

In [25]:
data.select("customer_id","pet_order_number","wet_food_order_number","orders_since_first_wet_trays_order","pet_food_tier").show(15, truncate=False) # 행이 길어도 자르지 않습니다.

+--------------------+----------------+---------------------+----------------------------------+-------------+
|customer_id         |pet_order_number|wet_food_order_number|orders_since_first_wet_trays_order|pet_food_tier|
+--------------------+----------------+---------------------+----------------------------------+-------------+
|10574848487411271014|2               |1.0                  |1.0                               |superpremium |
|10574848487411271014|1               |null                 |null                              |superpremium |
|10574848487411271014|8               |7.0                  |7.0                               |superpremium |
|10574848487411271014|4               |3.0                  |3.0                               |superpremium |
|10574848487411271014|9               |8.0                  |8.0                               |superpremium |
|6342772975217424401 |6               |null                 |null                              |premium      |
|

- 고객 id, 건식 주문 횟수, 습식 주문 횟수, 첫번째 습식 주문으로부터 지난 날짜(습식을 주문하지 않았으면 null 입니다),
- 주문한 사료의 등급을 골라서 보겠습니다.

In [19]:
data.describe("pet_order_number", "total_minutes_on_website_since_last_order").show()

+-------+-----------------+-----------------------------------------+
|summary| pet_order_number|total_minutes_on_website_since_last_order|
+-------+-----------------+-----------------------------------------+
|  count|            49042|                                    49042|
|   mean|3.506545410056686|                        92.71646751763794|
| stddev|2.864753860998993|                       291.26281915410124|
|    min|                1|                                        0|
|    max|               20|                                     8203|
+-------+-----------------+-----------------------------------------+



- describe()를 이용한 행의 정보입니다.
- 데이터셋에서 평균적으로 3.5번 주문을 했고, 총 49002개의 데이터가 있습니다. 최소 1번 이상 주문한 고객들만 모았네요.
- 마지막 주문 이후 웹사이트 방문기록은 평균 92.7분입니다. 최대 8203분(136.7시간) 동안 주문하지 않고 머무른 고객도 있네요. 이상 탐지가 필요할 듯 합니다. - 경쟁 기업은 아닐지?)
- 100분 이상 머무른 고객에게는 쿠폰을 발급하는 것도 좋겠네요.

In [24]:
data.describe("total_web_sessions_since_last_order", "total_minutes_on_website_since_last_order", "wet_trays").show()

+-------+-----------------------------------+-----------------------------------------+------------------+
|summary|total_web_sessions_since_last_order|total_minutes_on_website_since_last_order|         wet_trays|
+-------+-----------------------------------+-----------------------------------------+------------------+
|  count|                              49042|                                    49042|             49042|
|   mean|                 2.0364789364218425|                        92.71646751763794| 5.432139798540027|
| stddev|                  2.619177606705806|                       291.26281915410124|10.888737406010312|
|    min|                                  0|                                        0|                 0|
|    max|                                 51|                                     8203|               248|
+-------+-----------------------------------+-----------------------------------------+------------------+



- wet tray는 습식 캔의 갯수입니다. 구매한 캔의 갯수인지, 장바구니에 담긴 갯수인지는 캐글 사이트에 명확히 기재되어있지 않았습니다. 
- 해당 데이터가 없는 고객은 null로 count에 들어갔습니다.
- 평균 5.43개가 있습니다.

- 마지막 주문 이후 웹사이트 세션은 평균 2번입니다. 2번 방문 후 새로 구매했을 수도 있고, 아닐 수도 있습니다. (아직 미구매 고객의 경우)

In [23]:
data.groupby("pet_food_tier").count().orderBy("count", ascending=False).show()

+-------------+-----+
|pet_food_tier|count|
+-------------+-----+
| superpremium|28531|
|          mid|11518|
|      premium| 8993|
+-------------+-----+



- 사료 등급으로 주문을 묶어보았습니다. 가장 좋은 등급의 주문이 많았고, 2번째로 좋았던 등급의 주문이 가장 적었네요. 2위인 mid 등급과 1위인 super premium 등급의 차이가 크다는 것이 흥미롭습니다. 반려동물에게 좋은 것만 먹이고 싶다는 주인의 마음인 걸까요?

In [22]:
data.groupby("pet_breed_size").count().orderBy("count", ascending=False).show()

+--------------+-----+
|pet_breed_size|count|
+--------------+-----+
|         small|15751|
|        medium|14657|
|         large|10894|
|           toy| 6632|
|         giant| 1108|
+--------------+-----+



- 키우는 동물의 사이즈는 소형, 중형이 가장 많았습니다. 거대종은 수가 매우 적네요.

## 고객 필터링 - 할인쿠폰 or 푸시알림 대상

In [45]:
# 필터 1 - 마지막 주문 이후 2번 초과 웹사이트에 방문하고 습식을 주문하지 않은 고객
condition1 = (data.total_web_sessions_since_last_order > 2) & (data.wet_food_order_number.isNull())

# 필터 2 - 마지막 주문 이후 웹사이트에 100분 초과 체류한 고객
condition2 = data.total_minutes_on_website_since_last_order > 100

data2 = data.filter(condition1).filter(condition2)
data2.show()

+--------------------+--------------------+----------------+---------------------+----------------------------------+---------------------------+-------------+--------------------+--------------------+--------------------+---------------------+--------+------+--------------+---------------+----------------------+------------------------+-----------------------+-------------------+-----------+--------+----------------+---------+-------------------------+-------------+-------------------+------------------+--------------------------+------------------+-----------------------------------+------------------------+-----------------------------------------+----------------------+---------------------------------------+---------------------------------+--------------------------------+
|         customer_id|              pet_id|pet_order_number|wet_food_order_number|orders_since_first_wet_trays_order|pet_has_active_subscription|pet_food_tier| pet_signup_datetime|   pet_allergen_list|pet_fav_f

In [46]:
data2.count()

4011

- 49000여개의 데이터 중 4011개의 고객이 남았습니다.
- 이 고객을 대상으로 할인쿠폰 발행 & 푸시 알림 등을 하고 효과가 있는지 지켜보면 좋겠네요.

## 모델 빌딩

In [13]:
# integer 칼럼의 null을 0으로 대체
data = data.na.fill(value=0)

In [30]:
data.show()

+--------------------+--------------------+----------------+---------------------+----------------------------------+---------------------------+-------------+--------------------+-----------------+--------------------+---------------------+--------+------+--------------+---------------+----------------------+------------------------+-----------------------+-------------------+-----------+--------+----------------+---------+-------------------------+-------------+-------------------+------------------+--------------------------+------------------+-----------------------------------+------------------------+-----------------------------------------+----------------------+---------------------------------------+---------------------------------+--------------------------------+
|         customer_id|              pet_id|pet_order_number|wet_food_order_number|orders_since_first_wet_trays_order|pet_has_active_subscription|pet_food_tier| pet_signup_datetime|pet_allergen_list|pet_fav_flavour

In [14]:
from pyspark.sql.types import DoubleType # 예측을 Double 형태로 함 - 나중에 에러가 덜 난다고 합니다,,

data3 = data # shallow copy

# withColumn - 원래 컬럼 대체
data3 = data3.withColumn("pet_order_number",
                         data3["pet_order_number"].cast(DoubleType()))
data3 = data3.withColumn("orders_since_first_wet_trays_order",
                         data3["orders_since_first_wet_trays_order"].cast(DoubleType()))
data3 = data3.withColumn("total_web_sessions_since_last_order",
                         data3["total_web_sessions_since_last_order"].cast(DoubleType()))
data3 = data3.withColumn("total_minutes_on_website_since_last_order",
                         data3["total_minutes_on_website_since_last_order"].cast(DoubleType()))

건식 주문 횟수, 첫번째 습식 주문 후 주문 건수, 마지막 주문 후 웹 세션, 마지막 주문 후 웹사이트 체류 시간을 학습에 사용하고 습식을 주문할지를 예측해 보겠습니다.

### VectorAssembler를 사용해 모델 만들기

In [15]:
from pyspark.ml.feature import VectorAssembler

inputcols = ["pet_order_number", "orders_since_first_wet_trays_order", "total_web_sessions_since_last_order","total_minutes_on_website_since_last_order"]

assembler = VectorAssembler(inputCols = inputcols, outputCol = "predictors")

- 예측을 위해 predictor라는 컬럼을 새로 만듭니다.

In [16]:
predictors = assembler.transform(data3)

predictors.columns

['customer_id',
 'pet_id',
 'pet_order_number',
 'wet_food_order_number',
 'orders_since_first_wet_trays_order',
 'pet_has_active_subscription',
 'pet_food_tier',
 'pet_signup_datetime',
 'pet_allergen_list',
 'pet_fav_flavour_list',
 'pet_health_issue_list',
 'neutered',
 'gender',
 'pet_breed_size',
 'signup_promo',
 'ate_wet_food_pre_tails',
 'dry_food_brand_pre_tails',
 'pet_life_stage_at_order',
 'order_payment_date',
 'kibble_kcal',
 'wet_kcal',
 'total_order_kcal',
 'wet_trays',
 'wet_food_discount_percent',
 'wet_tray_size',
 'premium_treat_packs',
 'dental_treat_packs',
 'wet_food_textures_in_order',
 'total_web_sessions',
 'total_web_sessions_since_last_order',
 'total_minutes_on_website',
 'total_minutes_on_website_since_last_order',
 'total_wet_food_updates',
 'total_wet_food_updates_since_last_order',
 'last_customer_support_ticket_date',
 'customer_support_ticket_category',
 'predictors']

- input 데이터보다 더 많은 숫자의 컬럼이 들어가있네요. 데이터셋의 모든 컬럼을 포함합니다.

In [38]:
model_data = predictors.select("predictors", "pet_food_tier")

model_data.show(15)

+--------------------+-------------+
|          predictors|pet_food_tier|
+--------------------+-------------+
|  [2.0,1.0,4.0,32.0]| superpremium|
|   [1.0,0.0,1.0,3.0]| superpremium|
|   [8.0,7.0,0.0,0.0]| superpremium|
|  [4.0,3.0,6.0,15.0]| superpremium|
|   [9.0,8.0,1.0,0.0]| superpremium|
|   [6.0,0.0,2.0,1.0]|      premium|
|   [3.0,0.0,5.0,1.0]|      premium|
|   [9.0,0.0,6.0,4.0]|      premium|
|       (4,[0],[1.0])|      premium|
|   [7.0,0.0,5.0,9.0]|      premium|
|[11.0,0.0,6.0,813.0]|      premium|
|   [2.0,0.0,1.0,3.0]|      premium|
|   [8.0,0.0,6.0,3.0]|      premium|
|   [5.0,0.0,2.0,7.0]|      premium|
|  [4.0,0.0,4.0,74.0]|      premium|
+--------------------+-------------+
only showing top 15 rows



- 학습에 사용한 칼럼을 바탕으로 pet_food_tier (사료 등급) 을 예측합니다.
- 프리미엄 사료와 저급 사료를 얼마나 미리 주문할지에 대한 예측으로 사용할 수 있겠네요.

- 습식을 얼마나 주문할지도 예측해 보겠습니다.(전체 데이터 대상)

In [39]:
# 습식 주문횟수 예측
model_data = predictors.select("predictors", "wet_food_order_number")
model_data.show(15)

+--------------------+---------------------+
|          predictors|wet_food_order_number|
+--------------------+---------------------+
|  [2.0,1.0,4.0,32.0]|                  1.0|
|   [1.0,0.0,1.0,3.0]|                  0.0|
|   [8.0,7.0,0.0,0.0]|                  7.0|
|  [4.0,3.0,6.0,15.0]|                  3.0|
|   [9.0,8.0,1.0,0.0]|                  8.0|
|   [6.0,0.0,2.0,1.0]|                  0.0|
|   [3.0,0.0,5.0,1.0]|                  0.0|
|   [9.0,0.0,6.0,4.0]|                  0.0|
|       (4,[0],[1.0])|                  0.0|
|   [7.0,0.0,5.0,9.0]|                  0.0|
|[11.0,0.0,6.0,813.0]|                  0.0|
|   [2.0,0.0,1.0,3.0]|                  0.0|
|   [8.0,0.0,6.0,3.0]|                  0.0|
|   [5.0,0.0,2.0,7.0]|                  0.0|
|  [4.0,0.0,4.0,74.0]|                  0.0|
+--------------------+---------------------+
only showing top 15 rows



- 이 예측의 정확률이 높다면 습식을 어느 정도로 미리 주문해놓을지, 재고관리가 편리해질 것입니다.

In [41]:
# 학습, 테스트 데이터 8:2로 나누기

train_data, test_data = model_data.randomSplit([0.8, 0.2])

In [49]:
# 선형회귀 모델 사용
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(
    featuresCol = 'predictors', 
    labelCol = 'wet_food_order_number', 
    maxIter=10, regParam=0.3, elasticNetParam=0.8)

lrModel = lr.fit(train_data)

pred = lrModel.evaluate(test_data)

In [50]:
lrModel.coefficients
# 회귀계수 - 독립변수가 한 단위 변화함에 따라 종속변수에 미치는 영향력 크기
# 하지만 두 변수 사이에 상관관계가 없으면 회귀계수는 의미가 없게 된다.

DenseVector([0.0, 0.6227, 0.0, 0.0])

In [51]:
lrModel.intercept
# 절편

0.18303081409941796

In [52]:
pred.predictions.show(20)

+-------------+---------------------+-------------------+
|   predictors|wet_food_order_number|         prediction|
+-------------+---------------------+-------------------+
|(4,[0],[1.0])|                  0.0|0.18303081409941796|
|(4,[0],[1.0])|                  0.0|0.18303081409941796|
|(4,[0],[1.0])|                  0.0|0.18303081409941796|
|(4,[0],[1.0])|                  0.0|0.18303081409941796|
|(4,[0],[1.0])|                  0.0|0.18303081409941796|
|(4,[0],[1.0])|                  0.0|0.18303081409941796|
|(4,[0],[1.0])|                  0.0|0.18303081409941796|
|(4,[0],[1.0])|                  0.0|0.18303081409941796|
|(4,[0],[1.0])|                  0.0|0.18303081409941796|
|(4,[0],[1.0])|                  0.0|0.18303081409941796|
|(4,[0],[1.0])|                  0.0|0.18303081409941796|
|(4,[0],[1.0])|                  0.0|0.18303081409941796|
|(4,[0],[1.0])|                  0.0|0.18303081409941796|
|(4,[0],[1.0])|                  0.0|0.18303081409941796|
|(4,[0],[1.0])

- 아니.. 예측이 다 같네요. 0으로 찍은 듯 합니다.

### 랜덤포레스트 모델 사용

In [54]:
inputcols

['pet_order_number',
 'orders_since_first_wet_trays_order',
 'total_web_sessions_since_last_order',
 'total_minutes_on_website_since_last_order']

In [74]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, OneHotEncoderEstimator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

categoricalColumns = ['pet_health_issue_list', 'pet_breed_size']
stages = []

for categoricalCol in categoricalColumns:
  stringIndexer = StringIndexer(inputCol=categoricalCol, 
                                outputCol=categoricalCol + 'Index')
  encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
  stages += [stringIndexer, encoder]

numericCols  = ['pet_order_number', 'orders_since_first_wet_trays_order', 'wet_trays', 'total_web_sessions', 'total_wet_food_updates', 'total_web_sessions_since_last_order', 'total_minutes_on_website_since_last_order']
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=10)
stages += [rf]

# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=stages)

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "wet_food_order_number").show(10)


IllegalArgumentException: ignored

In [ ]:

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

rfModel = model.stages[2]
print(rfModel)  # summary only

In [75]:
from pyspark.ml.feature import StringIndexer, VectorIndexer, OneHotEncoderEstimator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

categoricalColumns = ['pet_health_issue_list', 'pet_breed_size']
stages = []

for categoricalCol in categoricalColumns:
  stringIndexer = StringIndexer(inputCol=categoricalCol, 
                                outputCol=categoricalCol + 'Index')
  encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
  stages += [stringIndexer, encoder]

numericCols  = ['pet_order_number', 'orders_since_first_wet_trays_order', 'wet_trays', 'total_web_sessions', 'total_wet_food_updates', 'total_web_sessions_since_last_order', 'total_minutes_on_website_since_last_order']
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [78]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(data)
df = pipelineModel.transform(data)
cols = df.columns
selectedCols = ['label', 'features'] + cols
df = df.select(selectedCols)
df.printSchema()

AnalysisException: ignored